In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import geohash
import random
import math
import time
import pytz
from datetime import datetime

In [11]:
### CONSTRUCT FINAL DATAFRAME FROM INPUT
### CONSTRUCT FINAL DATAFRAME FROM INPUT
### CONSTRUCT FINAL DATAFRAME FROM INPUT
### CONSTRUCT FINAL DATAFRAME FROM INPUT
### CONSTRUCT FINAL DATAFRAME FROM INPUT

def get_time_vector(hour, week):
    arr = numpy.zeros(168, dtype=float)
    arr[hour * week] = 1.0

def transform_dataset(df):
    def get_geohash_from_concat_latlon(latlon):
        lat, lon = latlon.split(":")
        lat = float(lat)
        lon = float(lon)        
        precision = 6  
        if (lat < -1000.0 or lat > 1000.0 or lon < -1000.0 or lon > 1000.0):
            return 0
        return geohash.encode(lat, lon, precision)

    tz = pytz.timezone('America/New_York')
    def get_local_time(timestamp):
        d = datetime.utcfromtimestamp(timestamp).replace(tzinfo=pytz.utc)
        dt = d.astimezone(tz)
        return dt

    def get_start_of_the_day(start_timestamp):
        return start_timestamp.replace(hour=0, minute=0, second=0)

    
    def get_hour_block(hour):
        if hour >= 0 and hour <= 3:
            return 0
        elif hour > 7 and hour <= 7
#     d["geo"] = d["start_geohash"] + ":" + d["end_geohash"]
    start_geo_1 = df["start_lat"].astype(str) + ":" + df["start_lng"].astype(str)
    start_geo = start_geo_1.apply(get_geohash_from_concat_latlon)
    start_geo.name = 'start_geohash'
    df = pd.concat([df, start_geo], axis = 1)    
    del start_geo_1

    end_geo_1 = df["end_lat"].astype(str) + ":" + df["end_lng"].astype(str)
    end_geo = end_geo_1.apply(get_geohash_from_concat_latlon)
    end_geo.name = 'end_geohash'
    df = pd.concat([df, end_geo], axis = 1)    
    del end_geo_1
    
    start_time = df["start_timestamp"].apply(get_local_time)
    start_time.name = 'start_timestamp_obj'
    df = pd.concat([df, start_time], axis = 1)    
    # start_time[:10]

    start = df["start_timestamp_obj"].apply(get_start_of_the_day)
    start.name = 'start_of_day'
    df = pd.concat([df, start], axis = 1)

    hour = df["start_timestamp_obj"].dt.hour
    hour.name = 'hour'
    df = pd.concat([df, hour], axis = 1)

    minute = df["start_timestamp_obj"].dt.minute
    minute.name = 'minute'
    df = pd.concat([df, minute], axis = 1)

    weekday = df["start_timestamp_obj"].dt.weekday
    weekday.name = 'weekday'
    df = pd.concat([df, weekday], axis = 1)

    df["geo"] = df["start_geohash"] + ":" + df["end_geohash"]
    
    timeseries = pd.Series(df["hour"], df["weekday"])
    del start_time
    del end_geo
    del hour
    del minute
    del weekday
    outdf = df.copy()
    outdf.drop('start_lng', axis=1, inplace=True)
    outdf.drop('start_lat', axis=1, inplace=True)
    outdf.drop('end_lng', axis=1, inplace=True)
    outdf.drop('end_lat', axis=1, inplace=True)
    outdf.drop('start_timestamp', axis=1, inplace=True)
    outdf.drop('start_timestamp_obj', axis=1, inplace=True)
    outdf.drop('start_of_day', axis=1, inplace=True)
    return outdf
#     df = pd.concat([df, pd.DataFrame(start_time, index=df.index)], axis = 1)
dfff = pd.read_csv("test.csv")
d = dfff[:100].copy()
d = transform_dataset(d)
d[:10]

,row_id,start_geohash,end_geohash,hour,minute,weekday,geo
0,0,dr5rsj,dr5rsp,3,13,6,dr5rsj:dr5rsp
1,1,dr5rus,dr5ruy,13,2,1,dr5rus:dr5ruy
2,2,dr5rvj,dr5rvn,13,2,1,dr5rvj:dr5rvn
3,3,dr5ru2,dr72jh,4,8,3,dr5ru2:dr72jh
4,4,dr5rug,dr5ruq,23,9,2,dr5rug:dr5ruq
5,5,dr5ru2,dr5rst,20,41,3,dr5ru2:dr5rst
6,6,dr5ruu,dr5ru0,22,37,5,dr5ruu:dr5ru0
7,7,dr5ru1,dr5ru5,11,58,5,dr5ru1:dr5ru5
8,8,dr5rvj,dr5ru6,23,22,2,dr5rvj:dr5ru6
9,9,dr5rus,dr5ruv,13,56,3,dr5rus:dr5ruv


In [ ]:
### GET training dataset
inputdf = pd.read_csv("train.csv")
df = inputdf
df = transform_dataset(df)
finalized_data = df

In [ ]:
df = transform_dataset(df)

In [5]:
# read the model from file 
inputdf = pd.read_csv("train_mod.csv")
df = inputdf
finalized_data = df

In [ ]:
### GET testing dataset
inputdf = pd.read_csv("train.csv")
df = inputdf
df = transform_dataset(df)
finalized_data = df

In [10]:
start_geohashes = df["start_geohash"]
end_geohashes = df["end_geohash"]
print len(start_geohashes) + len(end_geohashes)

25811430


In [ ]:
### GENERATE dictionary of geohashes and reverse_dictionary

from collections import defaultdict
dictionary = {}
reverse_dictionary = {}
start_list = list(df['start_geohash'])
end_list = list(df['end_geohash'])
geoset = set(start_list + end_list)

for i, item in enumerate(geoset):
    dictionary[i] = item
    reverse_dictionary[item] = i

tup = defaultdict(list)
def geo_mapping(combo):
    start, end, duration = combo.split(":")
    tup[start].append((int(duration), end))
    
# df["geo"] = df["start_geohash"] + ":" + df["end_geohash"] + ":" + df["duration"].astype(str)
_ = df["geo"].apply(geo_mapping)


sorted_tup = {}
for k, v in tup.iteritems():
    sorted_tup[k] = sorted(tup[k])[:300]

In [ ]:
### generate a data pipeline for the model input

In [ ]:
### Convert geohashes into a vector

In [ ]:
### may be add avg speed at the geohash-time as a feature?
### what about feature vector that is a combo of geohash/timewindow/
### write the input into separate files